In [11]:
import time
import datetime
import os # 用於處理檔案路徑
import fubon_neo

# 確保 fubon_neo 已經安裝
try:    
    from fubon_neo.sdk import FubonSDK, Order
    from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction
except ImportError:
    print("fubon_neo SDK 未安裝。請先執行 'pip install fubon-neo' 或按照富邦文件安裝。")
    exit()

# 連結 API Server
sdk = FubonSDK()


# 執行登入
print("嘗試登入富邦證券API...")
login_success = sdk.login("")   # 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
if login_success:
    print("登入成功！")
    print(accounts)
else:
    print("登入失敗，請檢查帳號、密碼或憑證資訊。")
    exit() # 登入失敗則結束程式

sdk.init_realtime() # 建立行情元件連線

# 取得股票行情資料的 REST Client
restStock = sdk.marketdata.rest_client.stock

# --- 設定監控參數 ---
PRICE_MIN = 50.0  # 您要篩選的最低股價
PRICE_MAX = 70.0  # 您要篩選的最高股價
TOP_VOLUME_SCAN_LIMIT = 200 # 初始抓取整體市場成交量前N名，再從中篩選 (設定較大數字以增加篩選命中率)
TARGET_TOP_COUNT = 10     # 篩選後，要抓取成交量前N名進行判斷

# --- 輔助函式：計算Tick大小 ---
def get_tick_size(price):
    """
    根據台灣股市常見的價格區間，回傳對應的tick大小。
    Args:
        price (float): 股票價格。
    Returns:
        float: 對應的tick大小。
    """
    if price < 10:
        return 0.01
    elif 10 <= price < 50:
        return 0.05
    elif 50 <= price < 100:
        return 0.1
    elif 100 <= price < 500:
        return 0.5
    elif 500 <= price < 1000:
        return 1.0
    else: # price >= 1000
        return 5.0

# --- 通知函式 ---
def send_notification(message):
    """
    發送通知訊息。
    目前僅列印到控制台。您可以修改此函式來整合其他通知服務（如LINE Notify）。
    Args:
        message (str): 要發送的訊息內容。
    """
    print(f"【通知】{datetime.datetime.now().strftime('%H:%M:%S')} - {message}")
    # 範例：如果需要整合 LINE Notify，您可以在這裡加入相關程式碼
    # import requests
    # LINE_NOTIFY_TOKEN = "您的LINE Notify權杖"
    # headers = {"Authorization": f"Bearer {LINE_NOTIFY_TOKEN}"}
    # data = {"message": message}
    # requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)


# --- 主監控迴圈 ---
print(f"開始股票監控程式... 監控價格區間: {PRICE_MIN}~{PRICE_MAX}，並從中找出成交量前{TARGET_TOP_COUNT}支。")
try:
    while True:
        current_datetime = datetime.datetime.now()
        current_time = current_datetime.time()

        # 設定交易時間區間 (例如，台股開盤 09:00 到收盤 13:30)
        market_open_time = datetime.time(9, 0)
        market_close_time = datetime.time(13, 30)

        # 檢查是否在交易時間內
        """
        if not (market_open_time <= current_time <= market_close_time):
            print(f"目前時間 {current_time.strftime('%H:%M:%S')} 不在交易時間 ({market_open_time.strftime('%H:%M')} - {market_close_time.strftime('%H:%M')}) 內，程式將暫停或停止。")
            if current_time > market_close_time or current_time < market_open_time:
                # 簡單的休息策略：盤後休息較久
                print("等待下一個交易日開盤...")
                time.sleep(3600) # 休息一小時
            continue # 跳過本次迴圈的後續監控邏輯
        """    

        print(f"\n--- 執行股票檢查 (時間: {current_datetime.strftime('%Y-%m-%d %H:%M:%S')}) ---")

        # 1. 抓取整體市場成交量前 TOP_VOLUME_SCAN_LIMIT 支股票
        volume_movers_response = restStock.snapshot.actives(market='TSE', trade='volume')

        # print(volume_movers_response)
        
        qualified_stocks_in_range = [] # 用於存放符合價格區間的股票資料

        if volume_movers_response:
            # 取得原始的 TOP_VOLUME_SCAN_LIMIT 支股票
            #initial_top_stocks = volume_movers_response[:TOP_VOLUME_SCAN_LIMIT]
            
            print(f"從整體成交量前 {TOP_VOLUME_SCAN_LIMIT} 支中篩選價格區間 {PRICE_MIN}~{PRICE_MAX}...")

            for stock_info in volume_movers_response:
                symbol = stock_info['symbol']
                initial_volume = stock_info.get('trade_volume', 0) # 從actives結果中取得成交量

                # 取得該股票的即時報價，以確認其價格
                quote_response = restStock.intraday.quote(symbol=symbol)
                if quote_response:
                    current_price = quote_response.data.get('trade_price') # 最新成交價

                    if current_price is not None:
                        current_price = float(current_price) # 確保是數字

                        # 2. 判斷是否在指定的價格區間內
                        if PRICE_MIN <= current_price <= PRICE_MAX:
                            daily_high_price = float(quote_response.data.get('high_price', current_price)) # 當日最高價
                            
                            # 將符合條件的股票加入列表，包含必要的資訊用於後續判斷
                            qualified_stocks_in_range.append({
                                'symbol': symbol,
                                'current_price': current_price,
                                'daily_high_price': daily_high_price,
                                'volume': initial_volume # 使用從 actives 得到的成交量
                            })
                    # else:
                    #     print(f"警告: 無法取得 {symbol} 的即時價格或資料不完整。")
                # else:
                #     print(f"警告: 查詢 {symbol} 即時報價失敗: {quote_response.message}")
            
            # 3. 從符合價格區間的股票中，找出成交量前 TARGET_TOP_COUNT 支
            if qualified_stocks_in_range:
                # 依成交量降序排序
                sorted_qualified_stocks = sorted(qualified_stocks_in_range, key=lambda x: x['volume'], reverse=True)
                final_top_stocks_for_check = sorted_qualified_stocks[:TARGET_TOP_COUNT]

                print(f"在 {PRICE_MIN}~{PRICE_MAX} 區間內，成交量前 {TARGET_TOP_COUNT} 支股票: {[s['symbol'] for s in final_top_stocks_for_check]}")

                # 4. 對這些最終選定的股票進行價格判斷並發送通知
                for stock_data in final_top_stocks_for_check:
                    symbol = stock_data['symbol']
                    current_price = stock_data['current_price']
                    daily_high_price = stock_data['daily_high_price']

                    # 計算3個tick的價格
                    tick_size = get_tick_size(daily_high_price)
                    three_ticks_down_price = daily_high_price - (3 * tick_size)

                    message_to_send = "" # 初始化訊息

                    # 判斷並分類通知訊息
                    if current_price == daily_high_price:
                        # 情況3: 目前價格就是最高價
                        message_to_send = f"股票 {symbol}: 【情況3】目前價格 {current_price:.2f} 就是當日最高價！"
                    elif current_price >= three_ticks_down_price and current_price < daily_high_price:
                        # 情況2: 目前價格是最高價減3個tick價格 (或介於減3tick與最高價之間)
                        message_to_send = f"股票 {symbol}: 【情況2】目前價格 {current_price:.2f} 接近當日最高價 {daily_high_price:.2f} (高於或等於最高價減3個tick {three_ticks_down_price:.2f})"
                    elif current_price < three_ticks_down_price:
                        # 情況1: 目前價格小於最高價 (且低於最高價減3個tick)
                        message_to_send = f"股票 {symbol}: 【情況1】目前價格 {current_price:.2f} 小於當日最高價 {daily_high_price:.2f}，且已低於最高價減3個tick ({three_ticks_down_price:.2f})"
                    
                    # 發送通知
                    if message_to_send:
                        send_notification(message_to_send)
                    else:
                        print(f"股票 {symbol}: 目前價格 {current_price:.2f}，當日最高價 {daily_high_price:.2f} - 未符合特定通知條件。")
            else:
                print(f"在整體成交量前 {TOP_VOLUME_SCAN_LIMIT} 支中，未找到符合 {PRICE_MIN}~{PRICE_MAX} 價格區間的股票。")

        else:
            print(f"查詢整體成交量排行失敗或無資料: {volume_movers_response.message if not volume_movers_response.is_success else '無資料'}")

        print(f"本次檢查完成。等待30秒後進行下一次檢查...")
        time.sleep(30) # 等待30秒後進行下一次檢查

except KeyboardInterrupt:
    # 捕獲 Ctrl+C 中斷信號，優雅地結束程式
    print("\n偵測到鍵盤中斷 (Ctrl+C)。程式已停止。")
except Exception as e:
    # 捕獲其他所有未預期的錯誤
    print(f"\n發生未預期的錯誤: {e}")
    import traceback
    traceback.print_exc() # 列印完整的錯誤追蹤資訊，方便除錯
finally:
    # 確保在程式結束前登出SDK，釋放資源
    sdk.logout()
    print("已登出富邦SDK。")

嘗試登入富邦證券API...
登入成功！
Result {
  is_success: True,
  message: None,
  data: [Account {
    name: "張秉文",
    branch_no: "20117",
    account: "976793",
    account_type: "stock",
}]
}
開始股票監控程式... 監控價格區間: 50.0~70.0，並從中找出成交量前10支。

--- 執行股票檢查 (時間: 2025-07-15 17:38:50) ---
從整體成交量前 200 支中篩選價格區間 50.0~70.0...

發生未預期的錯誤: string indices must be integers, not 'str'
已登出富邦SDK。


Traceback (most recent call last):
  File "C:\Users\byron\AppData\Local\Temp\ipykernel_23156\915927854.py", line 116, in <module>
    symbol = stock_info['symbol']
             ~~~~~~~~~~^^^^^^^^^^
TypeError: string indices must be integers, not 'str'
